In [6]:
# import os

# def batch_rename(folder, prefix="img_", start_num=1):
#     files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
#     files.sort()  

#     num = start_num

#     for filename in files:
#         ext = os.path.splitext(filename)[1]  
#         new_name = f"{prefix}{num}{ext}"
#         src = os.path.join(folder, filename)
#         dst = os.path.join(folder, new_name)

#         os.rename(src, dst)
#         print(f"{filename} -> {new_name}")

#         num += 1
# folder_path = "/home/pearson/Projects/ECE253/test"
# batch_rename(folder_path, prefix="img_", start_num=193)

In [3]:
# import os
# from PIL import Image

# def convert_jpeg_to_png(folder_path, remove_original=True):
#     for filename in os.listdir(folder_path):
#         if filename.lower().endswith(('.jpg', '.jpeg')):
#             jpg_path = os.path.join(folder_path, filename)
#             png_path = os.path.join(
#                 folder_path,
#                 os.path.splitext(filename)[0] + '.PNG'
#             )
#             with Image.open(jpg_path) as img:
#                 img = img.convert('RGB') 
#                 img.save(png_path, 'PNG')

#             if remove_original:
#                 os.remove(jpg_path)

#             print(f"Converted: {filename} -> {os.path.basename(png_path)}")

# if __name__ == "__main__":
#     folder = "/home/pearson/Projects/ECE253/dataset" 
#     convert_jpeg_to_png(folder)

## CLAHE

### function:

In [19]:
import os
import glob
import cv2

def clahe_enhance_image(input_path, output_path,
                        clip_limit=4.0,
                        tile_grid_size=(8, 8)):
    img = cv2.imread(input_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f"[WARN] Cannot read image: {input_path}")
        return False

    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)

    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)
    cl = clahe.apply(l)

    merged = cv2.merge((cl, a, b))
    output = cv2.cvtColor(merged, cv2.COLOR_LAB2BGR)

    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    cv2.imwrite(output_path, output)
    print(f"[OK] {input_path} -> {output_path}")
    return True


def batch_clahe(input_root, output_root,
                exts=("*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"),
                clip_limit=4.0,
                tile_grid_size=(8, 8)):
    img_paths = []
    for ext in exts:
        img_paths.extend(glob.glob(os.path.join(input_root, "**", ext), recursive=True))

    print(f"Found {len(img_paths)} images")

    for in_path in img_paths:
        rel_path = os.path.relpath(in_path, input_root)
        out_path = os.path.join(output_root, rel_path)

        clahe_enhance_image(
            in_path,
            out_path,
            clip_limit=clip_limit,
            tile_grid_size=tile_grid_size
        )

### Run:

In [20]:
clahe_input_root = "../dataset"
clahe_output_root = "../results/CLAHE"

batch_clahe(clahe_input_root, clahe_output_root,
            clip_limit=4.0,
            tile_grid_size=(8, 8))

Found 169 images
[OK] ../dataset/img_1.PNG -> ../results/CLAHE/img_1.PNG
[OK] ../dataset/img_2.PNG -> ../results/CLAHE/img_2.PNG
[OK] ../dataset/img_3.PNG -> ../results/CLAHE/img_3.PNG
[OK] ../dataset/img_4.PNG -> ../results/CLAHE/img_4.PNG
[OK] ../dataset/img_5.PNG -> ../results/CLAHE/img_5.PNG
[OK] ../dataset/img_6.PNG -> ../results/CLAHE/img_6.PNG
[OK] ../dataset/img_7.PNG -> ../results/CLAHE/img_7.PNG
[OK] ../dataset/img_8.PNG -> ../results/CLAHE/img_8.PNG
[OK] ../dataset/img_9.PNG -> ../results/CLAHE/img_9.PNG
[OK] ../dataset/img_10.PNG -> ../results/CLAHE/img_10.PNG
[OK] ../dataset/img_11.PNG -> ../results/CLAHE/img_11.PNG
[OK] ../dataset/img_12.PNG -> ../results/CLAHE/img_12.PNG
[OK] ../dataset/img_13.PNG -> ../results/CLAHE/img_13.PNG
[OK] ../dataset/img_14.PNG -> ../results/CLAHE/img_14.PNG
[OK] ../dataset/img_15.PNG -> ../results/CLAHE/img_15.PNG
[OK] ../dataset/img_16.PNG -> ../results/CLAHE/img_16.PNG
[OK] ../dataset/img_17.PNG -> ../results/CLAHE/img_17.PNG
[OK] ../dataset

## ZeroDCE

### function:

In [21]:
import torch
import torch.nn as nn
import torchvision
import torch.backends.cudnn as cudnn
import torch.optim
import os
import sys
import argparse
import time
from ZeroDCE import dataloader, model
# import model
import numpy as np
from torchvision import transforms
from PIL import Image
import glob
import time


def zerodce_lowlight(image_path, in_root, out_root, net, scale_factor=12, device="cuda"):
    data_lowlight = Image.open(image_path).convert("RGB")
    data_lowlight = np.asarray(data_lowlight) / 255.0
    data_lowlight = torch.from_numpy(data_lowlight).float()

    h = (data_lowlight.shape[0] // scale_factor) * scale_factor
    w = (data_lowlight.shape[1] // scale_factor) * scale_factor
    data_lowlight = data_lowlight[0:h, 0:w, :]
    data_lowlight = data_lowlight.permute(2, 0, 1).unsqueeze(0).to(device)

    start = time.time()
    enhanced_image, params_maps = net(data_lowlight)
    end_time = time.time() - start
    print("Inference time: {:.4f}s".format(end_time))

    rel_path = os.path.relpath(image_path, in_root)
    result_path = os.path.join(out_root, rel_path)

    os.makedirs(os.path.dirname(result_path), exist_ok=True)

    torchvision.utils.save_image(enhanced_image, result_path)
    print("Saved to:", result_path)

    return end_time

### Run:

In [22]:
in_dir = "../dataset"
out_dir = "../results/ZeroDCE"
zerodce_model_weights_path = './ZeroDCE/trained_model/Epoch99.pth'
    
with torch.no_grad():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print("Using device:", device)

    scale_factor = 12

    os.makedirs(out_dir, exist_ok=True)

    DCE_net = model.enhance_net_nopool(scale_factor).to(device)
    DCE_net.load_state_dict(torch.load(zerodce_model_weights_path, map_location=device))
    DCE_net.eval()

    exts = ["*.jpg", "*.jpeg", "*.png", "*.JPG", "*.JPEG", "*.PNG"]
    test_list = []
    for ext in exts:
        test_list.extend(glob.glob(os.path.join(in_dir, ext)))

    sum_time = 0.0

    for image in test_list:
        print("Processing:", image)
        sum_time += zerodce_lowlight(image, in_dir, out_dir, DCE_net, scale_factor=scale_factor, device=device)

    print("Total time:", sum_time)
    print("Average per image:", sum_time / max(len(test_list), 1))

Using device: cuda
Processing: ../dataset/img_1.PNG


/tmp/ipykernel_4065148/606138989.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  DCE_net.load_state_dict(torch.load(zerodce_model_weights_path, map_location=device))


Inference time: 0.0064s
Saved to: ../results/ZeroDCE/img_1.PNG
Processing: ../dataset/img_2.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_2.PNG
Processing: ../dataset/img_3.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_3.PNG
Processing: ../dataset/img_4.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_4.PNG
Processing: ../dataset/img_5.PNG
Inference time: 0.0015s
Saved to: ../results/ZeroDCE/img_5.PNG
Processing: ../dataset/img_6.PNG
Inference time: 0.0013s
Saved to: ../results/ZeroDCE/img_6.PNG
Processing: ../dataset/img_7.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_7.PNG
Processing: ../dataset/img_8.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_8.PNG
Processing: ../dataset/img_9.PNG
Inference time: 0.0012s
Saved to: ../results/ZeroDCE/img_9.PNG
Processing: ../dataset/img_10.PNG
Inference time: 0.0014s
Saved to: ../results/ZeroDCE/img_10.PNG
Processing: ../dataset/img_11.PNG
Inference time: 0.0023s
Saved to: ../

## NAFNet

### Process:

```bash
cd NAFNet
```

run test.ipynb

## Wiener filter

In [ ]:
import os
import re
import cv2
import numpy as np


def wiener_deblur(img, kernel, K=0.01):
    # img: 2D 单通道
    img_fft = np.fft.fft2(img)
    kernel_fft = np.fft.fft2(kernel, s=img.shape[:2])
    kernel_fft_conj = np.conj(kernel_fft)
    wiener_filter = kernel_fft_conj / (kernel_fft * kernel_fft_conj + K)
    result_fft = img_fft * wiener_filter
    result = np.abs(np.fft.ifft2(result_fft))
    return result


def motion_kernel(length=15, angle=0):
    kernel = np.zeros((length, length))
    kernel[length // 2, :] = 1
    M = cv2.getRotationMatrix2D((length / 2, length / 2), angle, 1)
    kernel = cv2.warpAffine(kernel, M, (length, length))
    kernel /= kernel.sum()
    return kernel


def natural_key(s):

    return [int(text) if text.isdigit() else text.lower()
            for text in re.split(r'(\d+)', s)]


def process_image_file(in_path, out_path, kernel, K=0.01):
    img = cv2.imread(in_path, cv2.IMREAD_COLOR)
    if img is None:
        print(f'[WARN] Failed to read image: {in_path}')
        return


    b, g, r = cv2.split(img)
    b_res = wiener_deblur(b, kernel, K)
    g_res = wiener_deblur(g, kernel, K)
    r_res = wiener_deblur(r, kernel, K)


    result = cv2.merge([
        np.clip(b_res, 0, 255).astype(np.uint8),
        np.clip(g_res, 0, 255).astype(np.uint8),
        np.clip(r_res, 0, 255).astype(np.uint8),
    ])

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    cv2.imwrite(out_path, result)


def batch_wiener_deblur(
    input_dir,
    output_dir,
    length=3,
    angle=0,
    K=0.01,
    exts=('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')
):
    kernel = motion_kernel(length=length, angle=angle)

    all_imgs = []
    for root, _, files in os.walk(input_dir):
        for name in files:
            ext = os.path.splitext(name)[1].lower()
            if ext in exts:
                in_path = os.path.join(root, name)
                rel_path = os.path.relpath(in_path, input_dir)
                out_path = os.path.join(output_dir, rel_path)
                all_imgs.append((in_path, out_path))

    # all_imgs = sorted(all_imgs, key=lambda x: natural_key(x[0]))

    print(f'Found {len(all_imgs)} images in {input_dir}')

    for idx, (in_path, out_path) in enumerate(all_imgs):
        print(f'[{idx+1}/{len(all_imgs)}] {in_path} -> {out_path}')
        process_image_file(in_path, out_path, kernel, K)



In [50]:
input_dir = '/home/pearson/Projects/ECE253/results/ZeroDCE/'        
output_dir = '/home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/' 

batch_wiener_deblur(
    input_dir=input_dir,
    output_dir=output_dir,
    length=5,
    angle=0,
    K=0.001
)

print('All done.')

Found 169 images in /home/pearson/Projects/ECE253/results/ZeroDCE/
[1/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_1.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_1.PNG
[2/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_2.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_2.PNG
[3/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_3.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_3.PNG
[4/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_4.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_4.PNG
[5/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_5.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_5.PNG
[6/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_6.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_6.PNG
[7/169] /home/pearson/Projects/ECE253/results/ZeroDCE/img_7.PNG -> /home/pearson/Projects/ECE253/results/ZeroDCE_Wiener/img_7.PNG
[8/169] /home/pearson/P

In [51]:
input_dir = '/home/pearson/Projects/ECE253/results/CLAHE/'          
output_dir = '/home/pearson/Projects/ECE253/results/CLAHE_Wiener/'  

batch_wiener_deblur(
    input_dir=input_dir,
    output_dir=output_dir,
    length=5,
    angle=0,
    K=0.001
)

print('All done.')

Found 169 images in /home/pearson/Projects/ECE253/results/CLAHE/
[1/169] /home/pearson/Projects/ECE253/results/CLAHE/img_1.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_1.PNG
[2/169] /home/pearson/Projects/ECE253/results/CLAHE/img_2.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_2.PNG
[3/169] /home/pearson/Projects/ECE253/results/CLAHE/img_3.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_3.PNG
[4/169] /home/pearson/Projects/ECE253/results/CLAHE/img_4.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_4.PNG
[5/169] /home/pearson/Projects/ECE253/results/CLAHE/img_5.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_5.PNG
[6/169] /home/pearson/Projects/ECE253/results/CLAHE/img_6.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_6.PNG
[7/169] /home/pearson/Projects/ECE253/results/CLAHE/img_7.PNG -> /home/pearson/Projects/ECE253/results/CLAHE_Wiener/img_7.PNG
[8/169] /home/pearson/Projects/ECE253/results/CLAHE/i